In [79]:
import math
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


csv_trainingStations_mean_zscore = 'trainingStationsMissingNormZScore.csv'
df_trainingStations_mean_zscore = pd.read_csv(csv_trainingStations_mean_zscore)

csv_testStations_mean_zscore = 'testStationsMissingNormZScore.csv'
df_testStations_mean_zscore = pd.read_csv(csv_testStations_mean_zscore)

csv_testStations = 'testStations.csv'
df_testStations = pd.read_csv(csv_testStations)

In [80]:

cols_to_keep = ["station", "latitude", "longitude", "numDocks", "timestamp", "month", \
                "day", "hour", "weekday", "weekhour", "isHoliday", "temperature_C", "bikes_3h_ago", \
                "full_profile_3h_diff_bikes", "full_profile_bikes", \
                "short_profile_3h_diff_bikes", "short_profile_bikes", "bikes","Id"]
#df_trainingStations_mean_zscore.to_csv('X_train_pd.csv', index=False)

In [81]:
df_trainingStations_mean_zscore

,station,latitude,longitude,numDocks,timestamp,month,day,hour,weekday,weekhour,...,relHumidity_HR,airPressure_mb,precipitation_l_m2,bikes_3h_ago,full_profile_3h_diff_bikes,full_profile_bikes,short_profile_3h_diff_bikes,short_profile_bikes,bikes,Id
0,-1.073977,0.929882,-1.213803,-0.683290,-1.731867,-1.323280,-1.672743,-1.661580,-1.002326,-1.229603,...,-0.414667,0.418567,-0.063604,-0.097680,-0.932455,-0.459985,-0.704587,-0.859403,4,1
1,-1.073977,0.929882,-1.213803,-0.683290,-1.731588,-1.323280,-1.672743,-1.517109,-1.002326,-1.208933,...,0.074615,0.448996,-0.063604,-0.327360,-1.144962,-0.848615,-0.666024,-0.911014,7,2
2,-1.073977,0.929882,-1.213803,-0.683290,-1.731308,-1.323280,-1.672743,-1.372638,-1.002326,-1.188263,...,0.200352,0.361515,-0.063604,-0.327360,-1.274608,-0.947675,-0.588897,-0.807792,11,3
3,-1.073977,0.929882,-1.213803,-0.683290,-1.731028,-1.323280,-1.672743,-1.228168,-1.002326,-1.167593,...,0.074615,0.361515,-0.063604,-0.292409,-1.248491,-1.079329,-0.704587,-0.756181,10,4
4,-1.073977,0.929882,-1.213803,-0.683290,-1.730749,-1.323280,-1.672743,-1.083697,-1.002326,-1.146923,...,0.074615,0.274034,-0.063604,-0.097680,-1.226695,-1.432120,-0.704587,-0.807792,11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198115,1.108153,1.328760,0.890846,1.455762,1.730771,-0.146677,1.743359,1.083364,1.002690,1.106090,...,0.074615,-0.075890,-0.063604,2.084280,0.493395,0.047436,0.656538,0.290994,17,198116
198116,1.108153,1.328760,0.890846,1.455762,1.731051,-0.146677,1.743359,1.227835,1.002690,1.126760,...,0.137484,-0.075890,-0.063604,2.199120,0.493574,0.047327,0.658601,0.284552,0,198117
198117,1.108153,1.328760,0.890846,1.455762,1.731330,-0.146677,1.743359,1.372306,1.002690,1.147430,...,0.200352,0.011591,-0.063604,1.624920,0.493706,0.047218,0.658143,0.281484,0,198118
198118,1.108153,1.328760,0.890846,1.455762,1.731610,-0.146677,1.743359,1.516777,1.002690,1.168100,...,0.326090,0.011591,-0.063604,1.050720,0.493791,0.047109,0.656309,0.281178,0,198119


In [82]:
df_trainingStations_mean = df_trainingStations_mean_zscore[cols_to_keep]

In [83]:
#counter = 0
#mean_whole_col = df_trainingStations_mean[['temperature_C']].mean(skipna = True)
#for index, row in df_trainingStations_mean.iterrows():
#    if math.isnan(row['temperature_C']) == True:
#        df_trainingStations_mean.loc[counter,'temperature_C'] = mean_whole_col.values[0]
#    counter = counter + 1

In [84]:
X = df_trainingStations_mean
y = df_trainingStations_mean[["bikes","Id"]].to_numpy()
del X["bikes"]
X = X.to_numpy()

In [85]:
#Partimos el dataset de train en entrenamiento y test de forma aleatoria
X_train, X_test_cv, y_train, y_test_cv = train_test_split(X, y, random_state=1)

In [86]:
X_train_Id = X_train[:,17]
X_train = np.delete(X_train, 17, 1)
X_test_cv_Id = X_test_cv[:,17]
X_test_cv = np.delete(X_test_cv, 17, 1)
y_train_Id = y_train[:,1]
y_train = np.delete(y_train, 1, 1)
y_test_cv_Id = y_test_cv[:,1]
y_test_cv = np.delete(y_test_cv, 1, 1)

In [87]:
X_train

array([[-1.03501012, -0.18404404, -1.08471771, ..., -0.89400967,
        -1.24447644, -1.42712439],
       [-0.97656022, -1.61588593, -1.2827879 , ..., -0.51542382,
        -0.47320632,  0.53409472],
       [-1.01552682, -0.20533147, -0.80251719, ..., -0.7422179 ,
        -0.62746034, -0.44651483],
       ...,
       [-0.99604352, -0.14391198, -0.58769586, ..., -1.16360688,
        -0.28038879,  0.01798443],
       [-1.07397672,  0.92988219, -1.21380317, ..., -1.47131884,
        -0.89740489, -1.42712439],
       [ 0.95228653,  0.32755241,  0.85060093, ...,  0.85677154,
         0.64215128,  0.26651611]])

In [88]:
regr = LinearRegression().fit(X_train, y_train)

In [89]:
y_pred_cv = regr.predict(X_test_cv)

In [90]:
y_pred_cv = np.around(y_pred_cv.reshape(-1,1))
y_pred_cv = y_pred_cv.astype('int64')

In [91]:
print(metrics.mean_absolute_error(y_test_cv,y_pred_cv))

4.439168180900464


In [92]:
y_pred_cv_neg = pd.DataFrame(y_pred_cv, columns = ['AA'])
y_pred_cv_neg['AA'] = y_pred_cv_neg['AA'].apply(lambda x: 0 if x < 0 else x)
y_pred_cv_neg = y_pred_cv_neg.to_numpy() 

<b>A partir de aquí obtenemos los datos de TEST para introducirlos en el csv</b>

In [93]:
cols_to_keep = ["station", "latitude", "longitude", "numDocks", "timestamp", "month", \
                "day", "hour", "weekday", "weekhour", "isHoliday", "temperature_C", "bikes_3h_ago", \
                "full_profile_3h_diff_bikes", "full_profile_bikes", \
                "short_profile_3h_diff_bikes", "short_profile_bikes","Id"]

In [94]:
df_testStations_mean = df_testStations_mean_zscore[cols_to_keep]

In [95]:
df_testStations_mean_zscore_Id = df_testStations_mean['Id']

In [96]:
np_testStations_mean = df_testStations_mean.to_numpy()

In [97]:
del df_testStations_mean['Id']

In [98]:
y_pred_test = regr.predict(df_testStations_mean.to_numpy())

In [99]:
y_pred_test = y_pred_test.reshape(-1,1).astype('int64')

In [100]:
y_pred_test_Id = df_testStations_mean_zscore_Id.to_numpy().reshape(-1,1)

In [101]:
y_pred_test
y_pred_test_neg = pd.DataFrame(y_pred_test, columns = ['AA'])
y_pred_test_neg['AA'] = y_pred_test_neg['AA'].apply(lambda x: 0 if x < 0 else x)
y_pred_test_neg = y_pred_test_neg.to_numpy()

In [102]:
Train_CV_Result = np.concatenate((y_test_cv_Id.reshape(-1,1), y_pred_cv), axis=1)
Train_CV_Result = Train_CV_Result[np.argsort(Train_CV_Result[:, 0])]
Test_Result = np.concatenate((y_pred_test_Id, y_pred_test_neg), axis=1)

In [103]:
Test_Result = pd.DataFrame(Test_Result, columns=['Id','bikes'])
Train_CV_Result = pd.DataFrame(Train_CV_Result, columns=['Id','bikes'])

In [104]:
Neg_Train_CV_Result = Train_CV_Result
Neg_Test_Result = Test_Result

In [105]:
y_pred_cv_neg = Neg_Train_CV_Result['bikes']
y_pred_cv_neg = y_pred_cv_neg.to_numpy().reshape(-1,1)

In [106]:
Neg_Train_CV_Result.to_csv('Train_CV_Result_LR_Conc.csv', index=False)
Neg_Test_Result.to_csv('Test_Result_LR_Conc.csv', index=False)